In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#Step 1
Get line boxes and modify them

In [ ]:
IMG_DIR = "/content/drive/MyDrive/BTP/100Docbank"
DOCTR_ANNS_DIR = "/content/drive/MyDrive/BTP/100DocbankDoctrAnns"

In [ ]:
import os

ann_files = sorted([os.path.join(DOCTR_ANNS_DIR,i) for i in os.listdir(DOCTR_ANNS_DIR)])
len(ann_files)

100

In [ ]:
image_id_to_para_line = {i:[] for i in range(len(ann_files))}
len(image_id_to_para_line)

100

In [ ]:
import json
import bisect

for i,p in enumerate(ann_files):
    with open(p) as f:
        data=json.load(f)
        page = data["pages"][0]
        h,w = page["dimensions"]
        image_id_to_para_line[i].append((h,w))
        for block in page["blocks"]:
            for line in block["lines"]:
                line_bbox = line["geometry"]
                line_bbox = [line_bbox[0][0]*w, line_bbox[0][1]*h, line_bbox[1][0]*w, line_bbox[1][1]*h]
                image_id_to_para_line[i].append(line)
                        # break
    # break

In [ ]:
image_id_to_para_line[0]

[(2200, 1700),
 {'geometry': [[0.361328125, 0.1064453125], [0.6123046875, 0.123046875]],
  'words': [{'value': 'NLDSA',
    'confidence': 0.962926983833313,
    'geometry': [[0.361328125, 0.107421875], [0.419921875, 0.1220703125]]},
   {'value': 'model',
    'confidence': 0.9950376152992249,
    'geometry': [[0.421875, 0.107421875], [0.4697265625, 0.1220703125]]},
   {'value': 'for',
    'confidence': 0.9988906979560852,
    'geometry': [[0.4697265625, 0.1064453125], [0.49609375, 0.123046875]]},
   {'value': 'GRB',
    'confidence': 0.9999310970306396,
    'geometry': [[0.49609375, 0.107421875], [0.5361328125, 0.1220703125]]},
   {'value': 'afterglows',
    'confidence': 0.9987092018127441,
    'geometry': [[0.5380859375, 0.1083984375], [0.6123046875, 0.123046875]]}]},
 {'geometry': [[0.8935546875, 0.107421875], [0.908203125, 0.123046875]],
  'words': [{'value': '9',
    'confidence': 0.9998001456260681,
    'geometry': [[0.8935546875, 0.107421875], [0.908203125, 0.123046875]]}]},
 {'g

##Hide this

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_para_line[id]
    h,w = anns[0]
    for ann in anns[1:]:
        # label = ann[-1]
        box = ann["geometry"]
        box = (int(box[0][0]*w),int(box[0][1]*h),int(box[1][0]*w),int(box[1][1]*h))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        # draw.rectangle(box, outline="blue", width=1)
        for word in ann["words"]:
            word_box = (int(word["geometry"][0][0]*w),int(word["geometry"][0][1]*h),int(word["geometry"][1][0]*w),int(word["geometry"][1][1]*h))
            draw.rectangle(word_box, outline="green", width=1)
        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()
    # break

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
image_id_to_new_line = {i:[] for i in range(len(ann_files))}
len(image_id_to_new_line)

100

In [ ]:
X_THRESHOLD_1=60
X_THRESHOLD_2=8

In [ ]:
# xdiff_data = []
#format l:[(h,w), {geometry of line:(4 coordinates), word boxes: []}, .. more lines] (now scaled)
for k,v in image_id_to_para_line.items():
    h,w = v[0]
    # image_id_to_new_line[k].append(v[0])
    for line in v[1:]:
        if len(line["words"])==1:
            # print(k,"here")
            line_box = line["words"][0]["geometry"]
            line_box = [line_box[0][0]*w, line_box[0][1]*h, line_box[1][0]*w, line_box[1][1]*h]
            td = {"geometry": line_box, "words":[line_box]}
            image_id_to_new_line[k].append(td)
        else:
            word_bboxes=[]
            for word in line["words"]:
                word_bbox = word["geometry"]
                word_bbox = [word_bbox[0][0]*w, word_bbox[0][1]*h, word_bbox[1][0]*w, word_bbox[1][1]*h]
                word_bboxes.append(word_bbox)
            word_bboxes=sorted(word_bboxes)
            cur_left_x=-1
            cur_left_y=-1
            cur_right_x=-1
            cur_right_y=-1
            cur_word_bbox=[]

            for i in range(1,len(word_bboxes)):
                if cur_left_x<0 and cur_left_y<0:
                    cur_left_x=word_bboxes[i-1][0]
                    cur_left_y=word_bboxes[i-1][1]
                    cur_word_bbox.append(word_bboxes[i-1])

                cur_right_x=word_bboxes[i][2]
                cur_right_y=word_bboxes[i][3]

                if word_bboxes[i][0]-word_bboxes[i-1][0]>X_THRESHOLD_1 and word_bboxes[i][0]-word_bboxes[i-1][2]>X_THRESHOLD_2:
                    line_bbox = [cur_left_x, cur_left_y, word_bboxes[i-1][2], word_bboxes[i-1][3]]
                    td={"geometry":line_bbox, "words":cur_word_bbox}
                    image_id_to_new_line[k].append(td)
                    cur_left_x=word_bboxes[i][0]
                    cur_left_y=word_bboxes[i][1]
                    cur_word_bbox=[]
                
                cur_word_bbox.append(word_bboxes[i])
            
            # l=len(word_bboxes)
            line_bbox = [cur_left_x, cur_left_y, cur_right_x, cur_right_y]
            td={"geometry":line_bbox, "words":cur_word_bbox}
            image_id_to_new_line[k].append(td)
    # break

In [ ]:
image_id_to_new_line[0]

In [ ]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from google.colab.patches import cv2_imshow

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    print(imgp)
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_new_line[id]
    h,w = anns[0]
    for ann in anns[1:]:
        # label = ann[-1]
        box = ann["geometry"]
        box = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        # draw.rectangle(box, outline="blue", width=3)
        for word in ann["words"]:
            word_box = (int(word[0]),int(word[1]),int(word[2]),int(word[3]))
            draw.rectangle(word_box, outline="green", width=1)

        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_new_line[id]
    h,w = anns[0]
    for ann in anns[1:]:
        # label = ann[-1]
        box = ann["geometry"]
        box = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        draw.rectangle(box, outline="blue", width=1)
        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_new_line[id]
    h,w = anns[0]
    for ann in anns[1:]:
        # label = ann[-1]
        box = ann["geometry"]
        box = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        draw.rectangle(box, outline="blue", width=1)
        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()

Output hidden; open in https://colab.research.google.com to view.

##Clustering

In [70]:
from typing import List
import numpy as np

class Cluster(object):
    def __init__(self, values, **kwargs):
        self._values = sorted(values)
        self._np_values = np.array(self._values)
        self._mean = np.mean(self._np_values, axis=0)
        self._assess_func = kwargs["func"]
        self.__func_params = kwargs["params"]

    def __len__(self):
        return len(self._values)

    @property
    def mean(self):
        return self._mean

    @property
    def min(self):
        return self._values[0]

    @property
    def max(self):
        return self._values[-1]

    def __contains__(self, item):
        return (self.max - self._threshold) < item < (
                    self.min + self._threshold)

    def add(self, box):
        self._values.append(box)
        self._np_values = np.append(self._np_values, [box], axis=0)
        self._mean = np.mean(self._np_values, axis=0)

    def pop(self):
        self._values = self._values[1:]
        self._np_values = self._np_values[1:]
        self._mean = np.mean(self._np_values, axis=0)

    def assess(self, word_box):
        return self._assess_func(word_box,prev=self._values[-1], **self.__func_params)


def cluster_positions(line_boxes, **kwargs):
    # greedily cluster positions
    # cluster_positions([1, 4, 6], 4) will return two clusters
    # with [1, 4], and [6] due its greedy nature
    # xdiff_data = []
#format l:[(h,w), {geometry of line:(4 coordinates), word boxes: []}, .. more lines] (now scaled)
    h,w = line_boxes[0]
    clusters = []
    mappings = {}
    # image_id_to_new_line[k].append(v[0])
    for line in line_boxes[1:]:
        word_bboxes=[]
        
        for word in line["words"]:
            word_bbox = word["geometry"]
            word_bbox = [word_bbox[0][0]*w, word_bbox[0][1]*h, word_bbox[1][0]*w, word_bbox[1][1]*h]
            word_bboxes.append(word_bbox)
        word_bboxes=sorted(word_bboxes)

        first_word_box = word_bboxes[0]
        clusters.append(Cluster([first_word_box], **kwargs))
        mappings[tuple(first_word_box)] = 0

        for word_bbox in word_bboxes[1:]:
            if not clusters[-1].assess(word_bbox):
                clusters[-1].add(word_bbox)
            else:
                clusters.append(Cluster([word_bbox], **kwargs))
            mappings[tuple(word_bbox)] = len(clusters)-1

    return clusters, mappings

    # else:
        



def get_margins(clusters: List[Cluster], min_occurances: int):
    for c in clusters:
        if len(c) >= min_occurances:
            return c
    return clusters[0]

Cluster Approach 1: Seperate line boxes in 2 column cases

In [ ]:
def two_col_cluster(box,  **kwargs):
    box_1 = kwargs["prev"]
    x_threshold_1 = kwargs["x_threshold_1"]
    x_threshold_2 = kwargs["x_threshold_2"]
    return (box[0]-box_1[0]>x_threshold_1) and (box[0]-box_1[2]>x_threshold_2)

In [ ]:
image_id_to_new_line = {k:[] for k in image_id_to_para_line}
len(image_id_to_new_line)

100

In [ ]:
X_THRESHOLD_1=60
X_THRESHOLD_2=20

In [ ]:
# xdiff_data = []
#format l:[(h,w), {geometry of line:(4 coordinates), word boxes: []}, .. more lines] (now scaled)
for k,v in image_id_to_para_line.items():
    clusters, mappings = cluster_positions(v, func=two_col_cluster, params={"x_threshold_1":X_THRESHOLD_1, "x_threshold_2":X_THRESHOLD_2}, standard=False)
    image_id_to_new_line[k].append(v[0])
    image_id_to_new_line[k].append({})
    for c in clusters:
        word_boxes = c._values
        line_box = [word_boxes[0][0], word_boxes[0][1], word_boxes[-1][2], word_boxes[-1][3]]
        image_id_to_new_line[k][1][tuple(line_box)] = word_boxes 
    # break

In [ ]:
import json
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from google.colab.patches import cv2_imshow

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_new_line[id]
    h,w = anns[0]
    for ann in anns[1]:
        # label = ann[-1]
        box = ann
        box = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        draw.rectangle(box, outline="blue", width=1)
        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
files = sorted([os.path.join(IMG_DIR,i) for i in os.listdir(IMG_DIR)])
np.random.seed(69)
ids = np.random.randint(0,len(files),10)
for id in ids:
    imgp = files[id]
    image = Image.open(imgp)
    image = image.convert("RGB")
    draw = ImageDraw.Draw(image, "RGBA")
    font = ImageFont.load_default()
    anns = image_id_to_new_line[id]
    h,w = anns[0]
    for ann in anns[1]:
        # label = ann[-1]
        box = ann
        box = (int(box[0]),int(box[1]),int(box[2]),int(box[3]))
        # print(box)
        # general_box = annotation['box']
        # draw.rectangle(general_box, outline=label2color[label], width=2)
        # draw.text((general_box[0] + 10, general_box[1] - 10), label, fill=label2color[label], font=font)
        # words = annotation['words']
        # for word in words:
        #     box = word['box']
        draw.rectangle(box, outline="blue", width=1)
        # draw.text((ann[0] + 10, ann[1] - 10), id_to_class[label], fill=label2color[label], font=font)
    open_cv_image = np.array(image) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    cv2_imshow(open_cv_image)
    # cv2.imwrite()

Output hidden; open in https://colab.research.google.com to view.

If a neighbouring word box has same label, give it text region

In [ ]:
def two_col_cluster_2(box,  **kwargs):
    box_1 = kwargs["prev"]
    x_threshold = kwargs["x_threshold"]
    # x_threshold_2 = kwargs["x_threshold_2"]
    return (box[0]-box_1[2]>x_threshold)

In [ ]:
X_THRESHOLD_3 = 20

In [ ]:
image_id_to_new_merged_line = {k:[] for k in image_id_to_para_line}
len(image_id_to_new_merged_line)

100

In [ ]:
for k,v in image_id_to_new_line.items():
    line_bboxes = []
    for ann in v[1]:
        line_bboxes.append(ann)

    clusters, mappings = cluster_positions(line_bboxes, func=two_col_cluster_2, params={"x_threshold":X_THRESHOLD_3}, standard=True)
    image_id_to_new_line[k].append(v[0])
    for c in clusters:
        old_line_boxes = c._values
        word_boxes = []
        for b in old_line_boxes:
            word_boxes.extend(v[1][b])
        line_box = [old_line_boxes[0][0],old_line_boxes[0][1], old_line_boxes[-1][2], old_line_boxes[-1][3]]
        image_id_to_new_line[k][1][tuple(line_box)] = word_boxes
    # break

ValueError: ignored

In [ ]:
A = [[1, 2, 3], [4, 5, 6],[7,8,9]]
A[1:]

[[4, 5, 6], [7, 8, 9]]